# XGB Classifier with Wrapper-Based FS (Lexical + Content)

Steven Sison | March 9, 2024

## Description

This document will be used to train a model using the reduced feature set obtain by using the wrapper-based method, forward feature selection. The model will be evaluated in terms of the usual metrics (accuracy, precision, F1-score, recall) as well as the training time. The model will also be stored for future evaluation purposes.

## Training the Model

### Preliminaries

#### 1. Loading the Dataset

In [1]:
import pandas as pd                     # For data transformation
import numpy as numpy                   # For scientific calculations
import seaborn as sns                   # For data visualizations
import matplotlib.pyplot as plt         # For plotting
import plotly.graph_objects as go       # For plotting
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, ConfusionMatrixDisplay
from xgboost import XGBClassifier, DMatrix, train
from sklearn.pipeline import Pipeline
import time
from datetime import datetime
import joblib
import os

dataset = pd.read_csv("../../../02_feature-engineering/final-datasets/binary_unbalanced_with_lexicalcontent.csv")   

dataset.head()



,url_type,blank_lines_count,blank_spaces_count,word_count,average_word_len,webpage_size,webpage_entropy,js_count,sus_js_count,js_eval_count,...,has_swf_in_string,has_cgi_in_string,has_php_in_string,has_abuse_in_string,has_admin_in_string,has_bin_in_string,has_personal_in_string,has_update_in_string,has_verification_in_string,url_scheme
0,1,14,36,0,0.000000,1186,5.269303,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,415,3952,11450,4.683930,26155,4.565537,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,5715,88450,420,5.833333,339327,4.656704,13,10,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1461,6292,3049,4.515907,89249,5.149580,11,2,0,...,0,0,0,0,0,0,0,0,0,0
4,0,43,273,0,0.000000,1530,4.754726,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
dataset.head()

,url_type,blank_lines_count,blank_spaces_count,word_count,average_word_len,webpage_size,webpage_entropy,js_count,sus_js_count,js_eval_count,...,has_swf_in_string,has_cgi_in_string,has_php_in_string,has_abuse_in_string,has_admin_in_string,has_bin_in_string,has_personal_in_string,has_update_in_string,has_verification_in_string,url_scheme
0,1,14,36,0,0.000000,1186,5.269303,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,415,3952,11450,4.683930,26155,4.565537,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,5715,88450,420,5.833333,339327,4.656704,13,10,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1461,6292,3049,4.515907,89249,5.149580,11,2,0,...,0,0,0,0,0,0,0,0,0,0
4,0,43,273,0,0.000000,1530,4.754726,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
x_train, x_test, y_train, y_test = train_test_split(dataset.drop(columns=['url_type']), dataset['url_type'], test_size = 0.2, random_state=42)

#### 2. Preprocessing (Balancing)

In [4]:
dataset['url_type'].value_counts()

url_type
0    120080
1     62334
Name: count, dtype: int64

#### 3. Removing Unnecessary Features

In [5]:
important_features_wrapper_25 = ['blank_lines_count', 
                                 'word_count', 
                                 'js_count', 
                                 'js_link_count', 
                                 'js_winopen_count', 
                                 'meta_tag_count', 
                                 'anchor_tag_count', 
                                 'input_tag_count', 
                                 'has_free_in_html', 
                                 'url_length', 
                                 'url_domain_entropy', 
                                 'url_number_of_digits', 
                                 'url_path_length', 
                                 'url_host_length', 
                                 'url_number_of_subdirectories', 
                                 'url_has_port', 
                                 'url_number_of_fragments', 
                                 'url_num_periods', 
                                 'has_ebayisapi_in_string', 
                                 'has_logon_in_string', 
                                 'has_signon_in_string', 
                                 'has_loginasp_in_string', 
                                 'has_exe_in_string', 
                                 'has_zip_in_string', 
                                 'has_paypal_in_string']


important_features_wrapper_65 = ['blank_lines_count', 
                                 'word_count', 
                                 'js_count', 
                                 'js_find_count', 
                                 'js_link_count', 
                                 'js_winopen_count', 
                                 'title_tag_presence', 
                                 'meta_tag_count', 
                                 'anchor_tag_count', 
                                 'applet_tag_count', 
                                 'input_tag_count', 
                                 'has_free_in_html', 
                                 'has_access_in_html', 
                                 'url_length', 
                                 'url_ip_in_domain', 
                                 'url_domain_entropy', 
                                 'url_is_digits_in_domain', 
                                 'url_number_of_digits', 
                                 'url_string_entropy', 
                                 'url_path_length', 
                                 'url_host_length', 
                                 'url_number_of_subdirectories', 
                                 'url_num_subdomain', 
                                 'url_has_port', 
                                 'url_number_of_fragments', 
                                 'url_num_periods', 
                                 'url_num_equal', 
                                 'url_num_open_parenthesis', 
                                 'url_num_close_parenthesis', 
                                 'url_num_ampersand', 
                                 'url_num_at', 
                                 'has_account_in_string', 
                                 'has_webscr_in_string', 
                                 'has_ebayisapi_in_string', 
                                 'has_signin_in_string', 
                                 'has_banking_in_string', 
                                 'has_confirm_in_string', 
                                 'has_logon_in_string', 
                                 'has_signon_in_string', 
                                 'has_loginasp_in_string', 
                                 'has_loginphp_in_string', 
                                 'has_exe_in_string', 
                                 'has_zip_in_string', 
                                 'has_rar_in_string', 
                                 'has_jpg_in_string', 
                                 'has_gif_in_string', 
                                 'has_viewerphp_in_string', 
                                 'has_getImageasp_in_string', 
                                 'has_plugins_in_string', 
                                 'has_paypal_in_string', 
                                 'has_dbsysphp_in_string', 
                                 'has_configbin_in_string', 
                                 'has_downloadphp_in_string', 
                                 'has_payment_in_string', 
                                 'has_files_in_string', 
                                 'has_shopping_in_string', 
                                 'has_mailphp_in_string', 
                                 'has_jar_in_string', 
                                 'has_swf_in_string', 
                                 'has_cgi_in_string', 
                                 'has_php_in_string', 
                                 'has_abuse_in_string', 
                                 'has_bin_in_string', 
                                 'has_update_in_string', 
                                 'has_verification_in_string']


X_test_25 = x_test[important_features_wrapper_25]
X_train_25 = x_train[important_features_wrapper_25]

X_test_65 = x_test[important_features_wrapper_65]
X_train_65 = x_train[important_features_wrapper_65]

#### Hyper-parameter Tuning

##### 25 Features

In [6]:
import optuna
from sklearn.metrics import mean_squared_error # or any other metric
from sklearn.model_selection import train_test_split

# Define the objective function for Optuna
def objective_25(trial):
    # Define the search space for hyperparameters
    param = {
        'objective': 'binary:hinge',
        'eval_metric': 'error',
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'n_estimators': 100000, # Fix the boosting round and use early stopping
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 10.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),
        'lambda': trial.suggest_float('lambda', 0.1, 10.0),
        'alpha': trial.suggest_float('alpha', 0.0, 10.0),
    }
    
    # Split the data into further training and validation sets (three sets are preferable)
    train_data, valid_data, train_target, valid_target = train_test_split(X_train_25, y_train, test_size=0.2, random_state=42)
    
    # Convert the data into DMatrix format
    dtrain = DMatrix(train_data, label=train_target)
    dvalid = DMatrix(valid_data, label=valid_target)
    
    # Define the pruning callback for early stopping
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-error')
    
    # Train the model with early stopping
    model = train(param, dtrain, num_boost_round=100000, evals=[(dvalid, 'validation')], early_stopping_rounds=100, callbacks=[pruning_callback])
    
    # Make predictions on the test set
    dtest = DMatrix(valid_data)
    y_pred = model.predict(dtest)
    
    # Calculate the root mean squared error
    error = mean_squared_error(valid_target, y_pred, squared=False)
    
    return error

# Create an Optuna study and optimize the objective function
study_25 = optuna.create_study(direction='minimize')
study_25.optimize(objective_25, n_trials=100) # Control the number of trials

# Print the best hyperparameters and the best RMSE
best_params_25 = study_25.best_params
best_error_25 = study_25.best_value
print("Best Hyperparameters (25 Features): ", best_params_25)
print("Best Error (25 Features): ", best_error_25)

c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-04-09 10:38:56,072] A new study created in memory with name: no-name-a02da014-d70d-4ca7-9be6-218ecdc3918b
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [10:38:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-error:0.65995
[1]	validation-error:0.39148
[2]	validation-error:0.14017
[3]	validation-error:0.09857
[4]	validation-error:0.07980
[5]	validation-error:0.07120
[6]	validation-error:0.07007
[7]	validation-error:0.06732
[8]	validation-error:0.06595
[9]	validation-error:0.06150
[10]	validation-error:0.05948
[11]	validation-error:0.05838
[12]	validation-error:0.05701
[13]	validation-error:0.05598
[14]	validation-error:0.05468
[15]	validation-error:0.05427
[16]	validation-error:0.05215
[17]	validation-error:0.05129
[18]	validation-error:0.05030
[19]	validation-error:0.05043
[20]	validation-error:0.04968
[21]	validation-error:0.04923
[22]	validation-error:0.04923
[23]	validation-error:0.04910
[24]	validation-error:0.04821
[25]	validation-error:0.04797
[26]	validation-error:0.04783
[27]	validation-error:0.04790
[28]	validation-error:0.04735
[29]	validation-error:0.04721
[30]	validation-error:0.04690
[31]	validation-error:0.04666
[32]	validation-error:0.04646
[33]	validation-erro

[I 2024-04-09 10:38:57,954] Trial 0 finished with value: 0.21096472628488872 and parameters: {'eta': 0.2886318769335058, 'max_depth': 8, 'subsample': 0.5768667621948013, 'colsample_bytree': 0.5412279037809841, 'gamma': 7.842680867417257, 'min_child_weight': 0.47325622610216056, 'lambda': 9.55309499752866, 'alpha': 5.413530636240223}. Best is trial 0 with value: 0.21096472628488872.


[0]	validation-error:0.65995
[1]	validation-error:0.65995
[2]	validation-error:0.65995
[3]	validation-error:0.65995
[4]	validation-error:0.65995
[5]	validation-error:0.65995
[6]	validation-error:0.65995
[7]	validation-error:0.16024
[8]	validation-error:0.13619
[9]	validation-error:0.11810
[10]	validation-error:0.11618
[11]	validation-error:0.11557
[12]	validation-error:0.11529
[13]	validation-error:0.11584
[14]	validation-error:0.11611


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [10:38:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


...

[0]	validation-error:0.65995

c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [10:40:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)



[1]	validation-error:0.65995


[I 2024-04-09 10:40:51,449] Trial 99 pruned. Trial was pruned at iteration 2.


Best Hyperparameters (25 Features):  {'eta': 0.2591187766320547, 'max_depth': 10, 'subsample': 0.8941432769123177, 'colsample_bytree': 0.6716271958485475, 'gamma': 2.930264895577504, 'min_child_weight': 1.1018942908236469, 'lambda': 9.389749037239056, 'alpha': 0.9327478708285475}
Best Error (25 Features):  0.1901216176181182


##### 33 Features (Purely Lexical)

In [7]:
# Define the objective function for Optuna
def objective_65(trial):
    # Define the search space for hyperparameters
    param = {
        'objective': 'binary:hinge',
        'eval_metric': 'error',
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'n_estimators': 100000, # Fix the boosting round and use early stopping
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 10.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),
        'lambda': trial.suggest_float('lambda', 0.1, 10.0),
        'alpha': trial.suggest_float('alpha', 0.0, 10.0),
    }
    
    # Split the data into further training and validation sets (three sets are preferable)
    train_data, valid_data, train_target, valid_target = train_test_split(X_train_65, y_train, test_size=0.2, random_state=42)
    
    # Convert the data into DMatrix format
    dtrain = DMatrix(train_data, label=train_target)
    dvalid = DMatrix(valid_data, label=valid_target)
    
    # Define the pruning callback for early stopping
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-error')
    
    # Train the model with early stopping
    model = train(param, dtrain, num_boost_round=100000, evals=[(dvalid, 'validation')], early_stopping_rounds=100, callbacks=[pruning_callback])
    
    # Make predictions on the test set
    dtest = DMatrix(valid_data)
    y_pred = model.predict(dtest)
    
    # Calculate the root mean squared error
    error = mean_squared_error(valid_target, y_pred, squared=False)
    
    return error

# Create an Optuna study and optimize the objective function
study_65 = optuna.create_study(direction='minimize')
study_65.optimize(objective_65, n_trials=100) # Control the number of trials

# Print the best hyperparameters and the best RMSE
best_params_65 = study_65.best_params
best_error_65 = study_65.best_value
print("Best Hyperparameters (60 Features): ", best_params_65)
print("Best Error (60 Features): ", best_error_65)

[I 2024-04-09 10:40:51,471] A new study created in memory with name: no-name-17ba6c9f-abb9-4118-8798-a3b416ebc75e


[0]	validation-error:0.65995
[1]	validation-error:0.65995
[2]	validation-error:0.14832
[3]	validation-error:0.12228
[4]	validation-error:0.10631
[5]	validation-error:0.09545
[6]	validation-error:0.08829
[7]	validation-error:0.08661
[8]	validation-error:0.08511
[9]	validation-error:0.08209
[10]	validation-error:0.07983


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [10:40:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[11]	validation-error:0.07901
[12]	validation-error:0.07538
[13]	validation-error:0.07346
[14]	validation-error:0.07274
[15]	validation-error:0.06965
[16]	validation-error:0.06880
[17]	validation-error:0.06739
[18]	validation-error:0.06452
[19]	validation-error:0.06181
[20]	validation-error:0.06081
[21]	validation-error:0.05907
[22]	validation-error:0.05708
[23]	validation-error:0.05595
[24]	validation-error:0.05513
[25]	validation-error:0.05468
[26]	validation-error:0.05448
[27]	validation-error:0.05372
[28]	validation-error:0.05242
[29]	validation-error:0.05198
[30]	validation-error:0.05136
[31]	validation-error:0.05105
[32]	validation-error:0.05054
[33]	validation-error:0.05019
[34]	validation-error:0.05002
[35]	validation-error:0.04985
[36]	validation-error:0.04978
[37]	validation-error:0.04910
[38]	validation-error:0.04879
[39]	validation-error:0.04817
[40]	validation-error:0.04780
[41]	validation-error:0.04804
[42]	validation-error:0.04738
[43]	validation-error:0.04742
[44]	valid

[I 2024-04-09 10:40:53,456] Trial 0 finished with value: 0.20570211152802179 and parameters: {'eta': 0.25547428906195185, 'max_depth': 5, 'subsample': 0.787263871752651, 'colsample_bytree': 0.6635089321824321, 'gamma': 5.592719827883262, 'min_child_weight': 0.3187631062787594, 'lambda': 6.305901051635223, 'alpha': 5.742239270019741}. Best is trial 0 with value: 0.20570211152802179.


...

[4]	validation-error:0.05300
[5]	validation-error:0.05156
[6]	validation-error:0.04882
[7]	validation-error:0.04742
[8]	validation-error:0.04632
[9]	validation-error:0.04529
[10]	validation-error:0.04492
[11]	validation-error:0.04409
[12]	validation-error:0.04324
[13]	validation-error:0.04283
[14]	validation-error:0.04224
[15]	validation-error:0.04207
[16]	validation-error:0.04146
[17]	validation-error:0.04074
[18]	validation-error:0.04067
[19]	validation-error:0.04057
[20]	validation-error:0.04005
[21]	validation-error:0.04015
[22]	validation-error:0.04002
[23]	validation-error:0.03995
[24]	validation-error:0.03988
[25]	validation-error:0.03947
[26]	validation-error:0.03944
[27]	validation-error:0.03930
[28]	validation-error:0.03902
[29]	validation-error:0.03916
[30]	validation-error:0.03902
[31]	validation-error:0.03875
[32]	validation-error:0.03878
[33]	validation-error:0.03878
[34]	validation-error:0.03885


[I 2024-04-09 10:43:10,363] Trial 98 pruned. Trial was pruned at iteration 34.


[0]	validation-error:0.65995
[1]	validation-error:0.20992
[2]	validation-error:0.07517
[3]	validation-error:0.06493


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [10:43:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-04-09 10:43:10,771] Trial 99 pruned. Trial was pruned at iteration 3.


Best Hyperparameters (60 Features):  {'eta': 0.2917823306846722, 'max_depth': 10, 'subsample': 0.9877779465008263, 'colsample_bytree': 0.6472493929579971, 'gamma': 0.743354920954372, 'min_child_weight': 0.977590771327246, 'lambda': 8.69669967650715, 'alpha': 4.840006987262439}
Best Error (60 Features):  0.18528451784814443


In [8]:
print(best_params_25)
print(best_params_65)

{'eta': 0.2591187766320547, 'max_depth': 10, 'subsample': 0.8941432769123177, 'colsample_bytree': 0.6716271958485475, 'gamma': 2.930264895577504, 'min_child_weight': 1.1018942908236469, 'lambda': 9.389749037239056, 'alpha': 0.9327478708285475}
{'eta': 0.2917823306846722, 'max_depth': 10, 'subsample': 0.9877779465008263, 'colsample_bytree': 0.6472493929579971, 'gamma': 0.743354920954372, 'min_child_weight': 0.977590771327246, 'lambda': 8.69669967650715, 'alpha': 4.840006987262439}


#### Model Training

In [9]:
from sklearn.model_selection import KFold
import optuna

# Initialize CV
cv = KFold(n_splits=10, shuffle=True, random_state=1)

best_params_25['objective'] = 'binary:hinge'
best_params_25['eval_metric'] = 'error'

best_params_65['objective'] = 'binary:hinge'
best_params_65['eval_metric'] = 'error'

# Convert the data into DMatrix format
dtrain_25 = DMatrix(X_train_25, label=y_train)
dvalid_25 = DMatrix(X_test_25, label=y_test)

dtrain_65 = DMatrix(X_train_65, label=y_train)
dvalid_65 = DMatrix(X_test_65, label=y_test)

# Train the Model
xgb_classifier_25 = train(best_params_25, dtrain_25, num_boost_round=3000)
y_pred_25 = xgb_classifier_25.predict(dvalid_25)

print("Model with 12 Features Done.")

xgb_classifier_65 = train(best_params_65, dtrain_65, num_boost_round=3000)
y_pred_65 = xgb_classifier_65.predict(dvalid_65)

print("Model with 33 Features Done.")

Model with 12 Features Done.
Model with 33 Features Done.


#### Evaluation

In [10]:
# Classification Report
print(classification_report(y_test, y_pred_25))
print(classification_report(y_test, y_pred_65))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97     23942
           1       0.96      0.94      0.95     12541

    accuracy                           0.96     36483
   macro avg       0.96      0.96      0.96     36483
weighted avg       0.96      0.96      0.96     36483

              precision    recall  f1-score   support

           0       0.97      0.97      0.97     23942
           1       0.95      0.94      0.94     12541

    accuracy                           0.96     36483
   macro avg       0.96      0.95      0.96     36483
weighted avg       0.96      0.96      0.96     36483



In [11]:
'''# Confusion Matrix for 12 Features
cm_up = confusion_matrix(y_test, y_pred_12, labels=xgb_classifier_12.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm_up, display_labels = xgb_classifier_12.classes_)
disp.plot()
plt.show()'''

'# Confusion Matrix for 12 Features\ncm_up = confusion_matrix(y_test, y_pred_12, labels=xgb_classifier_12.classes_)\ndisp = ConfusionMatrixDisplay(confusion_matrix = cm_up, display_labels = xgb_classifier_12.classes_)\ndisp.plot()\nplt.show()'

In [12]:
'''# Cross Validation Score
scores = cross_val_score(XGBClassifier(**params_gbm),
                        X_train, y_train, scoring='accuracy', cv=cv).mean()

print(scores)'''

"# Cross Validation Score\nscores = cross_val_score(XGBClassifier(**params_gbm),\n                        X_train, y_train, scoring='accuracy', cv=cv).mean()\n\nprint(scores)"

In [13]:
# Dumping the model
joblib.dump(xgb_classifier_25, 'xgb_wrapper_25_lexical-content.sav')
joblib.dump(xgb_classifier_65, 'xgb_wrapper_65_lexical-content.sav')

['xgb_wrapper_65_lexical-content.sav']

In [14]:
import content_generator_25
import content_generator_65
import time

def xgb_predict_maliciousness_25(url):

    numerical_values = content_generator_25.feature_generator(url)
    # print(numerical_values)
    numerical_values = DMatrix(numerical_values)

    match xgb_classifier_25.predict(numerical_values):
        case 0:
            return "Benign"
        case 1:
            return "Malware"
        case 2:
            return "Phishing"
        case 3:
            return "Defacement"
        
def xgb_predict_maliciousness_65(url):

    numerical_values = content_generator_65.feature_generator(url)
    # print(numerical_values)
    numerical_values = DMatrix(numerical_values)

    match xgb_classifier_65.predict(numerical_values):
        case 0:
            return "Benign"
        case 1:
            return "Malware"
        case 2:
            return "Phishing"
        case 3:
            return "Defacement"

url = "www.facebook.com/"
print("Current URL: "+url)

print("------------- Wrapper-Based (25 Features) -------------")
for i in range(15):
    start = time.perf_counter()
    prediction = xgb_predict_maliciousness_25(url)
    end = time.perf_counter()
    print("Trial "+str(i))
    print(prediction)
    print(end-start)

print("------------- Wrapper-Based (65 Features) -------------")
for i in range(15):
    start = time.perf_counter()
    prediction = xgb_predict_maliciousness_65(url)
    end = time.perf_counter()
    print("Trial "+str(i))
    print(prediction)
    print(end-start)

Current URL: www.facebook.com/
------------- Wrapper-Based (25 Features) -------------
Trial 0
Benign
0.467541899997741
Trial 1
Benign
0.5337568999966606
Trial 2
Benign
0.41254280001157895
Trial 3
Benign
0.4157662999932654
Trial 4
Benign
0.41804870002670214
Trial 5
Benign
0.40383789996849373
Trial 6
Benign
0.5325599999632686
Trial 7
Benign
0.42950199998449534
Trial 8
Benign
0.6240614000125788
Trial 9
Benign
0.531210400047712
Trial 10
Benign
0.5152539000264369
Trial 11
Benign
0.4622187999775633
Trial 12
Benign
0.7843356000375934
Trial 13
Benign
0.3928817000123672
Trial 14
Benign
1.0443634999683127
------------- Wrapper-Based (65 Features) -------------
Trial 0
Benign
0.4507177000050433
Trial 1
Benign
0.4774360000155866
Trial 2
Benign
0.7073819999932311
Trial 3
Benign
0.6858496000058949
Trial 4
Benign
0.4840974000399001
Trial 5
Benign
0.9041571999550797
Trial 6
Benign
0.5178882000036538
Trial 7
Benign
0.4234266999992542
Trial 8
Benign
0.45338630001060665
Trial 9
Benign
0.4683559999684803